In [28]:
# ✅ Install required packages
!pip install langgraph --quiet
!pip install requests --quiet

import requests
import traceback
from typing import TypedDict, List, Dict, Union
from langgraph.graph import StateGraph, END

# 🔐 Together API KEY
TOGETHER_API_KEY = "api key "
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.1"

# 🌐 Call Together API
def together_chat(context, question):
    headers = {
        "Authorization": f"Bearer {TOGETHER_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": MODEL_NAME,
        "max_tokens": 1000,
        "temperature": 0.7,
        "messages": [
            {"role": "user", "content": f"Data:\n{context}\n\nQuestion: {question}"}
        ]
    }
    response = requests.post("https://api.together.xyz/v1/chat/completions", headers=headers, json=payload)
    response_json = response.json()

    if "error" in response_json:
        raise ValueError(f"Together API Error: {response_json['error']}")

    return response_json['choices'][0]['message']['content']

# 🧠 Agent state definition
class AgentState(TypedDict):
    user_query: str
    sub_tasks: List[str]
    results: List[Dict[str, Union[str, Dict]]]
    task_index: int
    done: bool

# 🧠 PlanAgent: break query into subtasks
def plan_agent(user_query):
    print("\n🎯 Planning Subtasks")
    output = together_chat("", f"Break this user query into 2 to 5 atomic subtasks: {user_query}")
    tasks = [line.strip("-• ") for line in output.strip().split("\n") if line.strip()]
    print("📌 Subtasks:")
    for i, task in enumerate(tasks):
        print(f"[{i+1}] {task}")
    return tasks

# 🛠 ToolAgent: Solve one task
def tool_agent(sub_task):
    print(f"\n🔧 Executing Task: {sub_task}")
    output = together_chat("", sub_task)
    if "```python" in output:
        try:
            code = output.split("```python")[1].split("```", 1)[0]
            local_vars = {}
            exec(code, {}, local_vars)
            print("✅ Output: Code executed successfully.")
            return {"tool_output": str(local_vars), "raw_response": output}
        except Exception as e:
            print("❌ Output: Code execution failed.")
            return {"tool_output": f"Execution Error: {e}\n{traceback.format_exc()}", "raw_response": output}
    print("✅ Output:", output.strip())
    return {"tool_output": output.strip(), "raw_response": output}

# 🔍 Reflection Agent: Validate and possibly modify/delete/add tasks
def reflection_agent(sub_task, tool_output):
    print(f"\n🪞 Reflecting on: {sub_task}")
    reflection = together_chat(
        f"Task: {sub_task}\nOutput: {tool_output}",
        "Was this solved correctly? Reply Yes or No and explain. If No, suggest Modify/Delete/Add and provide a modified task if needed."
    ).strip()
    print("💬 Reflection Summary:", reflection)

    action = "retry"
    modified_task = sub_task

    if "delete" in reflection.lower():
        action = "delete"
    elif "modify" in reflection.lower():
        action = "modify"
        lines = reflection.split("\n")
        for line in lines:
            if "modified task" in line.lower():
                modified_task = line.split(":", 1)[-1].strip()
                break
    elif "add" in reflection.lower():
        action = "add"
        new_tasks = [line.strip("-• ") for line in reflection.split("\n") if line.lower().startswith("new task")]
        return False, action, modified_task, new_tasks

    return ("yes" in reflection.lower()[:5]), action, modified_task, []

# 🔁 LangGraph Workflow
def langgraph_workflow():
    builder = StateGraph(AgentState)

    def plan_node(state: AgentState) -> AgentState:
        sub_tasks = plan_agent(state["user_query"])
        return {**state, "sub_tasks": sub_tasks, "results": [], "task_index": 0}

    def solve_node(state: AgentState) -> AgentState:
        sub_tasks = state["sub_tasks"]
        results = state["results"]
        index = state["task_index"]

        if index >= len(sub_tasks):
            return {**state, "done": True}

        current_task = sub_tasks[index]
        result = tool_agent(current_task)
        is_valid, action, modified_task, new_tasks = reflection_agent(current_task, result["tool_output"])

        if is_valid:
            print(f"🟢 Task [{index+1}] completed.")
            results.append({"task": current_task, "result": result["tool_output"]})
            index += 1
        else:
            if action == "modify":
                print(f"✏️ Modifying Task [{index+1}] to: {modified_task}")
                sub_tasks[index] = modified_task
            elif action == "delete":
                print(f"🗑️ Deleting Task [{index+1}]")
                sub_tasks.pop(index)
            elif action == "add":
                print(f"➕ Adding Tasks after [{index+1}]: {new_tasks}")
                sub_tasks = sub_tasks[:index+1] + new_tasks + sub_tasks[index+1:]
            else:
                print(f"🔁 Retrying Task [{index+1}]")

        return {
            **state,
            "sub_tasks": sub_tasks,
            "results": results,
            "task_index": index,
            "done": index >= len(sub_tasks)
        }

    builder.add_node("Plan", plan_node)
    builder.add_node("Solve", solve_node)
    builder.set_entry_point("Plan")
    builder.add_edge("Plan", "Solve")
    builder.add_conditional_edges("Solve", lambda s: END if s["done"] else "Solve")

    return builder.compile()

# 🚀 Run LangGraph Iteratively with User-Driven Modifications
query = "Analyze sentiment from a set of tweets and visualize results with a bar graph"
graph = langgraph_workflow()

initial_state: AgentState = {
    "user_query": query,
    "sub_tasks": [],
    "results": [],
    "task_index": 0,
    "done": False
}

state = initial_state
while True:
    state = graph.invoke(state)

    print("\n📦 Final Task Outputs:")
    for i, r in enumerate(state["results"]):
        print(f"\n[{i+1}] {r['task']}\n📝 Result:\n{r['result']}")

    action = input("\n🧠 Would you like to add / modify / delete any task? (type 'no' to finish): ").strip().lower()

    if action == "no":
        print("\n✅ Workflow complete.")
        break

    if action == "add":
        new_task = input("➕ Enter the new task to add: ").strip()
        state["sub_tasks"].append(new_task)
        state["done"] = False

    elif action == "modify":
        idx = int(input("✏️ Enter the task index to modify (1-based): ")) - 1
        new_task = input("🆕 Enter the modified task: ").strip()
        state["sub_tasks"][idx] = new_task
        state["done"] = False

    elif action == "delete":
        idx = int(input("🗑️ Enter the task index to delete (1-based): ")) - 1
        state["sub_tasks"].pop(idx)
        state["done"] = False

    state["task_index"] = 0
    state["results"] = []



🎯 Planning Subtasks
📌 Subtasks:
[1] 1. Collect a set of tweets
[2] 2. Analyze sentiment of the collected tweets
[3] 3. Create a bar graph representation of the sentiment analysis results
[4] 4. Display the bar graph
[5] 5. (Optional) Save the bar graph as an image file.

🔧 Executing Task: 1. Collect a set of tweets
✅ Output: Okay, here is a set of 10 tweets that I have collected:

1. "Just had the best sushi roll ever at @SushiRestaurant! The flavors were on point and the presentation was stunning. #foodie #sushi"
2. "Had an amazing time at the concert last night! The energy and excitement was palpable. Can't wait for the next one! #concert #music"
3. "Beautiful day for a hike! The scenery was breathtaking and the weather was perfect. #hiking #nature"
4. "Just finished reading @Author's book and I couldn't put it down! The characters were so relatable and the plot was gripping. Highly recommend! #book #reading"
5. "Looking forward to the weekend! There are so many fun events happening